In [1]:
pip install flask flask-cors requests pillow ibm-watsonx-ai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
import base64
import os
import tempfile
from PIL import Image
import io
import logging
from datetime import datetime
import traceback

# IBM Watson X AI imports
try:
    from ibm_watsonx_ai import Credentials
    from ibm_watsonx_ai.foundation_models import ModelInference
    WATSON_AVAILABLE = True
except ImportError:
    WATSON_AVAILABLE = False
    print("Warning: IBM Watson X AI not installed. Install with: pip install ibm-watsonx-ai")

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

In [ ]:
# IBM Watson X AI Configuration - Your predefined credentials
WATSONX_EU_APIKEY = "grsjZnkKVEfU6O3xITUQV1pur8cY8ZOLWdk6M9kT9LUb"
WATSONX_EU_PROJECT_ID = "3bb15573-631a-4c1d-9a24-41b1f50dc072"
WATSONX_URL = "https://us-south.ml.cloud.ibm.com"

In [5]:
class MedicalImageAnalyzer:
    def __init__(self):
        self.credentials = None
        self.models = {}
        self.initialize_watson()

    def initialize_watson(self):
        """Initialize Watson X AI credentials and models"""
        if not WATSON_AVAILABLE:
            logger.error("IBM Watson X AI not available")
            return False

        try:
            self.credentials = Credentials(
                url=WATSONX_URL,
                api_key=WATSONX_EU_APIKEY
            )

            # Initialize models
            self.models = {
                "pixtral": ModelInference(
                    model_id="mistralai/pixtral-12b",
                    credentials=self.credentials,
                    project_id=WATSONX_EU_PROJECT_ID,
                    params={"max_tokens": 300}
                ),
                "llama_vision": ModelInference(
                    model_id="meta-llama/llama-3-2-11b-vision-instruct",
                    credentials=self.credentials,
                    project_id=WATSONX_EU_PROJECT_ID,
                    params={"max_tokens": 300}
                )
            }

            logger.info("✅ Watson X AI models initialized successfully")
            return True

        except Exception as e:
            logger.error(f"❌ Failed to initialize Watson X AI: {str(e)}")
            return False

    def encode_image_from_file(self, image_file):
        """Convert uploaded file to base64 encoded string"""
        try:
            # Read image file
            image_data = image_file.read()

            # Validate and process image
            image = Image.open(io.BytesIO(image_data))

            # Convert to RGB if needed
            if image.mode in ('RGBA', 'P'):
                image = image.convert('RGB')

            # Resize if too large (max 2048x2048)
            max_size = (2048, 2048)
            if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
                image.thumbnail(max_size, Image.Resampling.LANCZOS)

            # Convert back to bytes
            img_buffer = io.BytesIO()
            image.save(img_buffer, format='JPEG', quality=85)
            img_buffer.seek(0)

            # Encode to base64
            encoded_image = base64.b64encode(img_buffer.getvalue()).decode('utf-8')

            logger.info(f"✅ Image processed successfully - Size: {image.size}")
            return encoded_image

        except Exception as e:
            logger.error(f"❌ Image processing failed: {str(e)}")
            raise Exception(f"Image processing failed: {str(e)}")

    def create_analysis_message(self, prompt, encoded_image):
        """Create message format for Watson X AI"""
        return [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"You are a medical AI assistant. Analyze this medical image and respond to: {prompt}. Provide a detailed, professional medical assessment."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encoded_image}"
                        }
                    }
                ]
            }
        ]

    def analyze_with_model(self, model_name, model, messages):
        """Analyze image with specific model"""
        try:
            logger.info(f"🔬 Starting analysis with {model_name}")

            response = model.chat(messages=messages)

            if response and 'choices' in response and len(response['choices']) > 0:
                result_text = response['choices'][0]['message']['content']
                logger.info(f"✅ {model_name} analysis completed")

                return {
                    'status': 'success',
                    'model_name': model_name,
                    'response': result_text,
                    'timestamp': datetime.now().isoformat()
                }
            else:
                raise Exception("Invalid response format from model")

        except Exception as e:
            logger.error(f"❌ {model_name} analysis failed: {str(e)}")
            return {
                'status': 'error',
                'model_name': model_name,
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }

    def analyze_image(self, image_file, prompt):
        """Main analysis function"""
        if not self.models:
            raise Exception("Watson X AI models not initialized")

        try:
            # Process image
            encoded_image = self.encode_image_from_file(image_file)

            # Create messages
            messages = self.create_analysis_message(prompt, encoded_image)

            # Analyze with both models
            results = {}

            for model_key, model in self.models.items():
                model_name = "Pixtral 12B" if model_key == "pixtral" else "Llama 3.2 11B Vision"
                results[model_key] = self.analyze_with_model(model_name, model, messages)

            return results

        except Exception as e:
            logger.error(f"❌ Analysis failed: {str(e)}")
            raise e

In [6]:
# Initialize analyzer
analyzer = MedicalImageAnalyzer()

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:httpx:HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-07-16&project_id=4fef3547-d708-43f0-b6a4-2deee63b75a9&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-07-16&project_id=4fef3547-d708-43f0-b6a4-2deee63b75a9&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
C:\Users\profe\AppData\Roaming\Python\Python310\site-packages\ibm_watsonx_ai\foundation_models\utils\utils.py:436: LifecycleWarning: Model 'mistralai/pixtral-12b' is in deprecated state from 2025-07-09 until 2025-10-08. IDs of alternative models: mistralai/mistral-small-3-1-24b-instruct-2503. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lif

In [7]:
@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'watson_available': WATSON_AVAILABLE,
        'models_initialized': bool(analyzer.models)
    })

In [8]:
@app.route('/api/analyze', methods=['POST'])
def analyze_medical_image():
    """Analyze medical image endpoint"""
    try:
        # Validate request
        if 'image' not in request.files:
            return jsonify({
                'status': 'error',
                'error': 'No image file provided'
            }), 400

        if 'prompt' not in request.form:
            return jsonify({
                'status': 'error',
                'error': 'No analysis prompt provided'
            }), 400

        image_file = request.files['image']
        prompt = request.form['prompt']

        # Validate file
        if image_file.filename == '':
            return jsonify({
                'status': 'error',
                'error': 'No image file selected'
            }), 400

        # Check file type
        allowed_extensions = {'png', 'jpg', 'jpeg', 'gif', 'bmp', 'webp'}
        file_extension = image_file.filename.rsplit('.', 1)[1].lower()

        if file_extension not in allowed_extensions:
            return jsonify({
                'status': 'error',
                'error': f'Unsupported file type. Allowed: {", ".join(allowed_extensions)}'
            }), 400

        logger.info(f"📤 Received analysis request: {prompt[:50]}...")

        # Analyze image
        results = analyzer.analyze_image(image_file, prompt)

        return jsonify({
            'status': 'success',
            'results': results,
            'prompt': prompt,
            'timestamp': datetime.now().isoformat()
        })

    except Exception as e:
        logger.error(f"❌ Analysis endpoint error: {str(e)}")
        logger.error(traceback.format_exc())

        return jsonify({
            'status': 'error',
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }), 500

In [9]:
@app.route('/api/models', methods=['GET'])
def get_models():
    """Get available models information"""
    models_info = {
        'pixtral': {
            'name': 'Pixtral 12B',
            'description': 'Mistral AI vision model specialized in image analysis',
            'capabilities': ['Medical imaging', 'General image analysis', 'Detailed descriptions']
        },
        'llama_vision': {
            'name': 'Llama 3.2 11B Vision',
            'description': 'Meta\'s vision-language model for comprehensive analysis',
            'capabilities': ['Medical assessment', 'Multi-modal understanding', 'Detailed reasoning']
        }
    }

    return jsonify({
        'status': 'success',
        'models': models_info,
        'available': bool(analyzer.models),
        'timestamp': datetime.now().isoformat()
    })

In [10]:
@app.errorhandler(413)
def too_large(e):
    return jsonify({
        'status': 'error',
        'error': 'File too large. Maximum size is 16MB.'
    }), 413

@app.errorhandler(500)
def internal_error(e):
    return jsonify({
        'status': 'error',
        'error': 'Internal server error occurred.'
    }), 500

In [11]:
if __name__ == '__main__':
    print("\n" + "="*60)
    print("🏥 HealthMate Medical Image Analysis API")
    print("="*60)
    print(f"📡 Server starting on http://localhost:5000")
    print(f"🔬 Watson X AI Available: {WATSON_AVAILABLE}")
    print(f"🤖 Models Initialized: {bool(analyzer.models)}")
    print("="*60)
    print("\n📋 Available Endpoints:")
    print("  GET  /api/health     - Health check")
    print("  POST /api/analyze    - Analyze medical image")
    print("  POST /api/chat       - Text-only chat")
    print("  GET  /api/models     - Get model information")
    print("\n🚀 Ready to analyze medical images!")
    print("="*60 + "\n")

    # Configure for production
    app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

    app.run(
        host='0.0.0.0',
        port=5000,
        debug=True,
        threaded=True
    )


🏥 HealthMate Medical Image Analysis API
📡 Server starting on http://localhost:5000
🔬 Watson X AI Available: True
🤖 Models Initialized: True

📋 Available Endpoints:
  GET  /api/health     - Health check
  POST /api/analyze    - Analyze medical image
  POST /api/chat       - Text-only chat
  GET  /api/models     - Get model information

🚀 Ready to analyze medical images!

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.164:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\profe\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 🏥 HealthMate Medical Image Analysis API

A Flask-based REST API that leverages IBM Watson X AI's vision models to analyze medical images. The system uses multiple AI models (Pixtral 12B and Llama 3.2 11B Vision) to provide comprehensive medical image analysis and health consultations.

## 🌟 Features

- **Multi-Model Analysis**: Uses both Mistral AI's Pixtral 12B and Meta's Llama 3.2 11B Vision models
- **Medical Image Processing**: Supports various image formats (PNG, JPG, JPEG, GIF, BMP, WebP)
- **Intelligent Image Preprocessing**: Automatic resizing, format conversion, and optimization
- **RESTful API**: Clean, well-documented endpoints for easy integration
- **Health Check Monitoring**: Built-in health monitoring and status endpoints
- **CORS Enabled**: Cross-origin resource sharing for web applications
- **Error Handling**: Comprehensive error handling and logging
- **File Size Management**: Configurable file size limits (16MB default)

## 🚀 Quick Start

### Prerequisites

- Python 3.8+
- IBM Watson X AI account and credentials
- Required Python packages (see Installation)

### Installation

1. **Clone the repository**
   ```bash
   git clone <repository-url>
   cd healthmate-api
   ```

2. **Install dependencies**
   ```bash
   pip install flask flask-cors requests pillow ibm-watsonx-ai
   ```

3. **Configure credentials**
   Update the Watson X AI credentials in the code:
   ```python
   WATSONX_EU_APIKEY = "your-api-key-here"
   WATSONX_EU_PROJECT_ID = "your-project-id-here"
   WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
   ```

4. **Run the application**
   ```bash
   python app.py
   ```

The API will be available at `http://localhost:5000`

## 📋 API Endpoints

### 🔍 Health Check
```http
GET /api/health
```
Returns server status and model availability.

**Response:**
```json
{
  "status": "healthy",
  "timestamp": "2024-08-04T10:30:00.000Z",
  "watson_available": true,
  "models_initialized": true
}
```

### 🔬 Analyze Medical Image
```http
POST /api/analyze
```
Analyzes medical images using both AI models.

**Parameters:**
- `image` (file): Medical image file
- `prompt` (string): Analysis question/prompt

**Example Request:**
```bash
curl -X POST \
  http://localhost:5000/api/analyze \
  -F "image=@chest_xray.jpg" \
  -F "prompt=What abnormalities can you detect in this chest X-ray?"
```

**Response:**
```json
{
  "status": "success",
  "results": {
    "pixtral": {
      "status": "success",
      "model_name": "Pixtral 12B",
      "response": "Analysis result from Pixtral model...",
      "timestamp": "2024-08-04T10:30:00.000Z"
    },
    "llama_vision": {
      "status": "success",
      "model_name": "Llama 3.2 11B Vision",
      "response": "Analysis result from Llama model...",
      "timestamp": "2024-08-04T10:30:00.000Z"
    }
  },
  "prompt": "What abnormalities can you detect in this chest X-ray?",
  "timestamp": "2024-08-04T10:30:00.000Z"
}
```

### 💬 Chat Endpoint
```http
POST /api/chat
```
Text-based health consultation endpoint.

**Request Body:**
```json
{
  "message": "What are the symptoms of pneumonia?"
}
```

**Response:**
```json
{
  "status": "success",
  "response": "Health guidance and recommendations...",
  "timestamp": "2024-08-04T10:30:00.000Z"
}
```

### 🤖 Models Information
```http
GET /api/models
```
Returns information about available AI models.

**Response:**
```json
{
  "status": "success",
  "models": {
    "pixtral": {
      "name": "Pixtral 12B",
      "description": "Mistral AI vision model specialized in image analysis",
      "capabilities": ["Medical imaging", "General image analysis", "Detailed descriptions"]
    },
    "llama_vision": {
      "name": "Llama 3.2 11B Vision",
      "description": "Meta's vision-language model for comprehensive analysis",
      "capabilities": ["Medical assessment", "Multi-modal understanding", "Detailed reasoning"]
    }
  },
  "available": true,
  "timestamp": "2024-08-04T10:30:00.000Z"
}
```

## 🖼️ Supported Image Formats

- **PNG** - Portable Network Graphics
- **JPG/JPEG** - Joint Photographic Experts Group
- **GIF** - Graphics Interchange Format
- **BMP** - Bitmap Image File
- **WebP** - Web Picture format

## ⚙️ Configuration

### File Size Limits
```python
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB
```

### Image Processing
- **Maximum Resolution**: 2048x2048 pixels
- **Auto-conversion**: RGBA and P mode images converted to RGB
- **Compression**: JPEG quality set to 85%
- **Format**: All images converted to JPEG for processing

### AI Model Parameters
```python
params = {"max_tokens": 300}
```

## 🔧 Development

### Running in Development Mode
```bash
python app.py
```
The server runs with:
- **Host**: 0.0.0.0 (accessible from any IP)
- **Port**: 5000
- **Debug**: True
- **Threading**: Enabled

### Jupyter Notebook
For development and testing, you can use the provided Jupyter notebook version:
1. Open `healthmate_notebook.ipynb`
2. Run cells sequentially
3. Test individual components

### Logging
The application uses Python's logging module with INFO level. Logs include:
- Request processing
- Image analysis progress  
- Error tracking
- Model performance

### Project Structure
```
healthmate-api/
├── app.py                 # Main Flask application
├── healthmate_notebook.ipynb  # Jupyter notebook version
├── README.md             # This file
├── requirements.txt      # Python dependencies
└── tests/               # Test files (optional)
```

## 🛠️ Error Handling

The API handles various error scenarios:

### Common Errors
- **400**: Bad Request (missing image/prompt)
- **413**: File too large (>16MB)
- **500**: Internal server error

### Error Response Format
```json
{
  "status": "error",
  "error": "Error description",
  "timestamp": "2024-08-04T10:30:00.000Z"
}
```

## 🔐 Security Considerations

1. **API Keys**: Keep Watson X AI credentials secure
2. **File Validation**: Only accept valid image formats
3. **Size Limits**: Prevent large file uploads
4. **CORS**: Configure appropriately for production
5. **Input Sanitization**: Validate all user inputs

## 📊 Usage Examples

### Python Client Example
```python
import requests

# Health check
response = requests.get('http://localhost:5000/api/health')
print(response.json())

# Image analysis
with open('medical_image.jpg', 'rb') as f:
    files = {'image': f}
    data = {'prompt': 'Analyze this medical image'}
    response = requests.post('http://localhost:5000/api/analyze',
                           files=files, data=data)
    print(response.json())
```

### JavaScript/Web Example
```javascript
const formData = new FormData();
formData.append('image', imageFile);
formData.append('prompt', 'What do you see in this X-ray?');

fetch('/api/analyze', {
    method: 'POST',
    body: formData
})
.then(response => response.json())
.then(data => console.log(data));
```

### cURL Example
```bash
# Health check
curl http://localhost:5000/api/health

# Image analysis
curl -X POST \
  -F "image=@sample.jpg" \
  -F "prompt=Analyze this medical image" \
  http://localhost:5000/api/analyze

# Chat
curl -X POST \
  -H "Content-Type: application/json" \
  -d '{"message":"What are COVID-19 symptoms?"}' \
  http://localhost:5000/api/chat
```

## 🚀 Production Deployment

### Environment Variables
For production, use environment variables for sensitive data:
```bash
export WATSONX_API_KEY="your-api-key"
export WATSONX_PROJECT_ID="your-project-id"
export WATSONX_URL="your-watson-url"
```

### WSGI Server
Use a production WSGI server like Gunicorn:
```bash
pip install gunicorn
gunicorn -w 4 -b 0.0.0.0:5000 app:app
```

### Docker Deployment
```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
EXPOSE 5000
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:5000", "app:app"]
```

## 📝 Requirements

Create a `requirements.txt` file:
```txt
Flask==2.3.3
Flask-CORS==4.0.0
requests==2.31.0
Pillow==10.0.1
ibm-watsonx-ai==1.0.0
```

## 🤝 Contributing

1. Fork the repository
2. Create a feature branch
3. Make your changes
4. Add tests if applicable
5. Submit a pull request

## 📄 License

This project is licensed under the MIT License - see the LICENSE file for details.

## ⚠️ Disclaimer

This tool is for educational and research purposes only. It should not be used as a substitute for professional medical advice, diagnosis, or treatment. Always consult with qualified healthcare professionals for medical concerns.

## 📞 Support

For support and questions:
- Create an issue in the repository
- Check the documentation
- Review the API responses for error details

## 🔄 Version History

- **v1.0.0**: Initial release with dual model support
- Multi-format image support
- RESTful API implementation
- Comprehensive error handling

---

**Built with ❤️ using Flask and IBM Watson X AI**